# Understanding Memory in LLMs

In the previous Notebook, we successfully explored how OpenAI models can enhance the results from Azure Cognitive Search. 

However, we have yet to discover how to engage in a conversation with the LLM. With [Bing Chat](http://chat.bing.com/), for example, this is possible, as it can understand and reference the previous responses.

There is a common misconception that GPT models have memory. This is not true. While they possess knowledge, they do not retain information from previous questions asked to them.

In this Notebook, our goal is to illustrate how we can effectively "endow the LLM with memory" by employing prompts and context.

In [1]:
import os
import random
from langchain.chat_models import AzureChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain.embeddings import AzureOpenAIEmbeddings
from langchain.docstore.document import Document
from langchain.memory import CosmosDBChatMessageHistory

from IPython.display import Markdown, HTML, display  

def printmd(string):
    display(Markdown(string))

#custom libraries that we will use later in the app
from common.utils import (
    get_search_results,
    update_vector_indexes,
    model_tokens_limit,
    num_tokens_from_docs,
    num_tokens_from_string,
    get_answer,
)

from common.prompts import COMBINE_CHAT_PROMPT_TEMPLATE

from dotenv import load_dotenv
load_dotenv("credentials_my.env")

import logging

# Get the root logger
logger = logging.getLogger()
# Set the logging level to a higher level to ignore INFO messages
logger.setLevel(logging.WARNING)

In [2]:
# Set the ENV variables that Langchain needs to connect to Azure OpenAI
os.environ["OPENAI_API_VERSION"] = os.environ["AZURE_OPENAI_API_VERSION"]

### Let's start with the basics
Let's use a very simple example to see if the GPT model of Azure OpenAI have memory. We again will be using langchain to simplify our code 

In [3]:
QUESTION = "Tell me some use cases for reinforcement learning"
FOLLOW_UP_QUESTION = "Give me the main points of our conversation"

In [4]:
# Define model
MODEL = os.environ["GPT35TURBO16K-0613-16k"] # check the other possible values in <model_tokens_limit()> of common/utils.py
COMPLETION_TOKENS = 500
# Create an OpenAI instance
llm = AzureChatOpenAI(deployment_name=MODEL, temperature=0.5, max_tokens=COMPLETION_TOKENS)

In [5]:
# We create a very simple prompt template, just the question as is:
prompt = PromptTemplate(
    input_variables=["question"],
    template="{question}",
)

chain = LLMChain(llm=llm, prompt=prompt)

In [6]:
# Let's see what the GPT model responds
response = chain.run(QUESTION)
printmd(response)

1. Robotics: Reinforcement learning can be used to teach robots how to perform complex tasks such and grasping objects, navigating through obstacles, and interacting with the environment.

2. Autonomous vehicles: Reinforcement learning can be used to train self-driving cars to make decisions in real-time, such as navigating traffic, following traffic laws, and avoiding accidents.

3. Game playing: Reinforcement learning has been used to train AI agents to play complex games such as chess, Go, and video games, where the agent learns from trial and error to improve its performance.

4. Personalized recommendations: Reinforcement learning can be used to create personalized recommendation systems for products, movies, music, and other content based on user preferences and feedback.

5. Finance and trading: Reinforcement learning can be used to optimize trading strategies, manage investment portfolios, and predict stock prices based on historical data and market conditions.

6. Healthcare: Reinforcement learning can be used to optimize treatment plans for patients, predict disease progression, and personalize healthcare interventions based on individual patient data.

7. Resource management: Reinforcement learning can be used to optimize resource allocation in areas such as energy management, supply chain logistics, and inventory management.

8. Industrial automation: Reinforcement learning can be used to optimize processes in manufacturing, logistics, and other industrial settings to improve efficiency and reduce costs.

9. Natural language processing: Reinforcement learning can be used to train AI agents to understand and generate human language, such as in chatbots, virtual assistants, and language translation systems.

10. Adaptive control systems: Reinforcement learning can be used to develop adaptive control systems for complex processes such as power plants, chemical plants, and other industrial systems to optimize performance and reduce energy consumption.

In [7]:
#Now let's ask a follow up question
chain.run(FOLLOW_UP_QUESTION)

"1. We discussed the current project status and any potential roadblocks.\n2. We talked about the upcoming deadlines and how to prioritize tasks.\n3. We reviewed the team's progress and any areas that may need additional support.\n4. We discussed potential solutions for improving team collaboration and communication.\n5. We also talked about the next steps and action items for each team member."

As you can see, it doesn't remember what it just responded, sometimes it responds based only on the system prompt, or just randomly. This proof that the LLM does NOT have memory and that we need to give the memory as a a conversation history as part of the prompt, like this:

In [8]:
hist_prompt = PromptTemplate(
    input_variables=["history", "question"],
    template="""
                {history}
                Human: {question}
                AI:
            """
    )
chain = LLMChain(llm=llm, prompt=hist_prompt)

In [9]:
Conversation_history = """
Human: {question}
AI: {response}
""".format(question=QUESTION, response=response)

In [10]:
printmd(chain.run({"history":Conversation_history, "question": FOLLOW_UP_QUESTION}))

1. Reinforcement learning can be used in robotics for complex tasks such as grasping objects and navigating obstacles.
2. It can be used to train self-driving cars to make real-time decisions and avoid accidents.
3. Reinforcement learning is used in game playing to train AI agents to improve their performance through trial and error.
4. It can create personalized recommendation systems based on user preferences and feedback.
5. Reinforcement learning can optimize trading strategies, manage investment portfolios, and predict stock prices in finance and trading.
6. It can optimize treatment plans, predict disease progression, and personalize healthcare interventions in the healthcare industry.
7. Reinforcement learning can optimize resource allocation in areas such as energy management and supply chain logistics.
8. It can optimize processes in manufacturing and industrial settings to improve efficiency and reduce costs.
9. Reinforcement learning can be used in natural language processing to train AI agents to understand and generate human language.
10. It can develop adaptive control systems for complex processes to optimize performance and reduce energy consumption.

**Bingo!**, so we now know how to create a chatbot using LLMs, we just need to keep the state/history of the conversation and pass it as context every time

## Now that we understand the concept of memory via adding history as a context, let's go back to our GPT Smart Search engine

In [11]:
# Since Memory adds tokens to the prompt, we would need a better model that allows more space on the prompt
COMPLETION_TOKENS = 2000
llm = AzureChatOpenAI(deployment_name=MODEL, temperature=0.5, max_tokens=COMPLETION_TOKENS)
embedder = AzureOpenAIEmbeddings(deployment=os.environ["EMBEDDING_DEPLOYMENT"], chunk_size=1) 

In [12]:
index1_name = "cogsrch-index-files"
index2_name = "cogsrch-index-csv"
index3_name = "cogsrch-index-books-vector"
text_indexes = [index1_name, index2_name]
vector_indexes = [index+"-vector" for index in text_indexes] + [index3_name]
vector_indexes

['cogsrch-index-files-vector',
 'cogsrch-index-csv-vector',
 'cogsrch-index-books-vector']

In [13]:
%%time

# Search in text-based indexes first and update vector indexes
k=10 # Top k results per each text-based index
ordered_results = get_search_results(QUESTION, text_indexes, k=k, reranker_threshold=1, vector_search=False)
update_vector_indexes(ordered_search_results=ordered_results, embedder=embedder)

# Search in all vector-based indexes available
similarity_k = 5 # top results from multi-vector-index similarity search
ordered_results = get_search_results(QUESTION, vector_indexes, k=k, vector_search=True,
                                        similarity_k=similarity_k,
                                        query_vector = embedder.embed_query(QUESTION))
print("Number of results:", len(ordered_results))

Number of results: 5
CPU times: user 6.55 s, sys: 152 ms, total: 6.71 s
Wall time: 41.1 s


In [ ]:
# Uncomment the below line if you want to inspect the ordered results
ordered_results

In [ ]:
top_docs = []
for key,value in ordered_results.items():
    location = value["location"] if value["location"] is not None else ""
    top_docs.append(Document(page_content=value["chunk"], metadata={"source": location+os.environ['BLOB_SAS_TOKEN']}))
        
print("Number of chunks:",len(top_docs))

In [ ]:
# Comment / Uncomment the below line if you want to inspect the ordered top_docs
print(f"Nr. of elements in top_docs: {len(top_docs)}. Here they are:\n{top_docs}")

In [ ]:
# Calculate number of tokens of our docs
if(len(top_docs)>0):
    tokens_limit = model_tokens_limit(MODEL) # this is a custom function we created in common/utils.py
    prompt_tokens = num_tokens_from_string(COMBINE_CHAT_PROMPT_TEMPLATE) # this is a custom function we created in common/utils.py
    context_tokens = num_tokens_from_docs(top_docs) # this is a custom function we created in common/utils.py
    
    requested_tokens = prompt_tokens + context_tokens + COMPLETION_TOKENS
    
    chain_type = "map_reduce" if requested_tokens > 0.9 * tokens_limit else "stuff"  
    
    print("System prompt token count:",prompt_tokens)
    print("Max Completion Token count:", COMPLETION_TOKENS)
    print("Combined docs (context) token count:",context_tokens)
    print("--------")
    print("Requested token count:",requested_tokens)
    print("Token limit for", MODEL, ":", tokens_limit)
    print("Chain Type selected:", chain_type)
        
else:
    print("NO RESULTS FROM AZURE SEARCH")

## Using load_qa_with_sources_chain
The *get_answer* function in the next cell is equivalent to the next code, in case **chain_type = map_reduce**:

```
from langchain.chains.qa_with_sources import load_qa_with_sources_chain
from common.prompts import (COMBINE_PROMPT, COMBINE_QUESTION_PROMPT)

llm35 = AzureChatOpenAI(deployment_name="gpt-35-turbo", temperature=0.5, max_tokens=COMPLETION_TOKENS)
chain = load_qa_with_sources_chain(llm35, chain_type="map_reduce",
                                   combine_prompt = COMBINE_PROMPT,
                                   question_prompt = COMBINE_QUESTION_PROMPT,
                                   return_intermediate_steps = True)

answer = chain( {"input_documents": top_docs, "question": QUESTION, "language": "it"}, return_only_outputs=True)
```

In [ ]:
%%time
# Get the answer
response = get_answer(llm=llm, docs=top_docs, query=QUESTION, language="English", chain_type=chain_type)
printmd(response['output_text'])

And if we ask the follow up question:

In [ ]:
response = get_answer(llm=llm, docs=top_docs,  query=FOLLOW_UP_QUESTION, language="English", chain_type=chain_type)
printmd(response['output_text'])

You might get a different response from above, but it doesn't matter what response you get, it will be based on the context given, not on previous answers.

Until now we just have the same as the prior Notebook 03: results from Azure Search enhanced by OpenAI model, with no memory

**Now let's add memory to it:**

Reference: https://python.langchain.com/docs/modules/memory/how_to/adding_memory_chain_multiple_inputs

In [ ]:
# memory object, which is neccessary to track the inputs/outputs and hold a conversation.
memory = ConversationBufferMemory(memory_key="chat_history",input_key="question")

response = get_answer(llm=llm, docs=top_docs, query=QUESTION, language="English", chain_type=chain_type, 
                        memory=memory)
printmd(response['output_text'])

In [ ]:
# Now we add a follow up question:
response = get_answer(llm=llm, docs=top_docs, query=FOLLOW_UP_QUESTION, language="English", chain_type=chain_type, 
                      memory=memory)
printmd(response['output_text'])

In [ ]:
# Another follow up query
response = get_answer(llm=llm, docs=top_docs, query="Translate to Italian", language="Italian", chain_type=chain_type,  
                      memory=memory)
printmd(response['output_text'])

You might get a different answer on the above cell, and it is ok, this bot is not yet well configured to answer any question that is not related to its knowledge base, including salutations.

Let's check our memory to see that it's keeping the conversation

In [ ]:
memory.buffer

## Using CosmosDB as persistent memory

In previous cell we have added local RAM memory to our chatbot. However, it is not persistent, it gets deleted once the app user's session is terminated. It is necessary then to use a Database for persistent storage of each of the bot user conversations, not only for Analytics and Auditing, but also if we wisg to provide recommendations. 

Here we will store the conversation history into CosmosDB for future auditing purpose.
We will use a class in LangChain use CosmosDBChatMessageHistory, see [HERE](https://python.langchain.com/en/latest/_modules/langchain/memory/chat_message_histories/cosmos_db.html)

In [ ]:
# Create CosmosDB instance from langchain cosmos class.
cosmos = CosmosDBChatMessageHistory(
    cosmos_endpoint=os.environ['AZURE_COSMOSDB_ENDPOINT'],
    cosmos_database=os.environ['AZURE_COSMOSDB_NAME'],
    cosmos_container=os.environ['AZURE_COSMOSDB_CONTAINER_NAME'],
    connection_string=os.environ['AZURE_COMOSDB_CONNECTION_STRING'],
    session_id="Agent-Test-Session" + str(random.randint(1, 1000)),
    user_id="Agent-Test-User" + str(random.randint(1, 1000))
    )

# prepare the cosmosdb instance
cosmos.prepare_cosmos()

In [ ]:
# Create or Memory Object
memory = ConversationBufferMemory(memory_key="chat_history",input_key="question",chat_memory=cosmos)

In [ ]:
# Testing using our Question
response = get_answer(llm=llm, docs=top_docs, query=QUESTION, language="English", chain_type=chain_type, 
                        memory=memory)
printmd(response['output_text'])

In [ ]:
# Now we add a follow up question:
response = get_answer(llm=llm, docs=top_docs, query=FOLLOW_UP_QUESTION, language="English", chain_type=chain_type, 
                      memory=memory)
printmd(response['output_text'])

In [ ]:
# Another follow up query
response = get_answer(llm=llm, docs=top_docs, query="Translate to Italian", language="Italian", chain_type=chain_type,  
                      memory=memory)
printmd(response['output_text'])

Let's check our Azure CosmosDB to see the whole conversation


In [ ]:
#load message from cosmosdb
cosmos.load_messages()
cosmos.messages

![CosmosDB Memory](./images/cosmos-chathistory.png)

# Summary
##### Adding memory to our application allows the user to have a conversation, however this feature is not something that comes with the LLM, but instead, memory is something that we must provide to the LLM in form of context of the question.

We added persitent memory using CosmosDB.

We also can notice that the current chain that we are using is smart, but not that much. Although we have given memory to it, it searches for similar docs everytime, regardless of the input and it struggles to respond to prompts like: Hello, Thank you, Bye, What's your name, What's the weather and any other task that is not search in the knowledge base.


## <u>Important Note</u>:<br>
As we proceed, while all the code will remain compatible with GPT-3.5 models, we highly recommend transitioning to GPT-4. Here's why:

**GPT-3.5-Turbo** can be likened to a 7-year-old child. You can provide it with concise instructions, but it frequently struggles to follow them accurately. Additionally, its limited memory can make sustained conversations challenging.

**GPT-3.5-Turbo-16k** resembles the same 7-year-old, but with an increased attention span for longer instructions. However, it still faces difficulties accurately executing them about half the time.

**GPT-4** exhibits the capabilities of a 10-12-year-old child. It possesses enhanced reasoning skills and more consistently adheres to instructions. While its memory retention for instructions is moderate, it excels at following them.

**GPT-4-32k** is akin to the 10-12-year-old child with an extended memory. It comprehends lengthy sets of instructions and engages in meaningful conversations. Thanks to its robust memory, it offers detailed responses.

Understanding this analogy above will become clearer as you complete the final notebook.


# NEXT
We know now how to do a Smart Search Engine that can power a chatbot!! great!

But, does this solve all the possible scenarios that a virtual assistant will require?  **What about if the answer to the Smart Search Engine is not related to text, but instead requires to look into tabular data?** The next notebook explains and solves the tabular problem and the concept of Agents